# User-User Collaborative Filtering

We use groups of similar users to make recommendations for a given user.

![alt text](images/user-user-example.png)

Bob and Alice's ratings are highly correlated.

So we can use Alice's rating for Star Wars to predict Bob's rating for Star Wars.

## Average Rating

Recall that the average rating is computed across all users who have rated an item.

It does not consider similarity between users.

$$s(i,j) = \frac{\sum_{i' \in \Omega_j}{r_{i'j}}}{\mid \Omega_j \mid}$$

We can improve on this by weighting each rating, where weights are a function of user similarity.

## Weighted Ratings

We want to down weight ratings from users that I disagree with, and increase the weight for users that are very similar to me.

$$s(i,j) = \frac{\sum_{i' \in \Omega_j}{w_{ii'}r_{i'j}}}{\sum_{i' \in \Omega_j}{w_{ii'}}}$$

$$r_{i'j} = \text{rating that user i' gives to item j}$$
$$w_{ii'} = \text{weight between user i' and user i (high for similar users)}$$

## Deviation

Users can be optimistic or pessimistic, and in general rate movies on different scales.

We can determine a user's *bias* with their overall average rating score. Users that tend to rate movies optimistically will have a higher average than those who are pessimistic.

We then use the bias to compute a ratings deviation from the user average.

$$dev(i,j) = r(i,j) - \bar{r}_i ,\text{for a known rating}$$

In other words, we normalize ratings by subtracting user mean ratings.

Our goal now is to predict user *i* deviation for item *j*.

$$
\hat{dev}(i,j) = \frac{1}{\mid \Omega_j \mid}
\sum_{i' \in \Omega_j}{r(i',j) - \bar{r}_{i'}}
$$

We can then score each item *j* using the average rating for user *i* and the predicted deviation for user *i* and item *j*.

$$
s(i,j) = \bar{r}_i + \hat{dev}(i,j)
$$

We do not need to add the bias $\bar{r}_i$ back to the score, however, since it is constant across a given user. Instead, we can just sort items by their predicted deviations.

## Combining Weights and Deviations

Recall that we want to weight scores using user similarities. We can combine the weighting scheme with the deviations.

We are now using absolute value of weight to normalize scores because weights can be negative.

$$
s(i,j) = \bar{r}_i +
    \frac
        {\sum_{i' \in \Omega_j}{w_{ii'}(r_{i'j} - \bar{r}_{i'})}}
        {\sum_{i' \in \Omega_j}{\mid w_{ii'} \mid}}
$$

## Calculating the User Similarity Weights

We use Pearson correlation, but we only consider mutually rated items.

$$w_{ii'} = \frac
    {\sum_{j \in \psi_{ii'}}({r_{ij}-\bar{r}_i)(r_{i'j}-\bar{r}_{i'})}}
    {
        \sqrt{\sum_{j \in \psi_{ii'}}{(r_{ij}-\bar{r}_i)^2}}
        \sqrt{\sum_{j \in \psi_{ii'}}{(r_{i'j}-\bar{r}_{i'})^2}}
    }
$$

$$\psi_{i} = \text{set of movies that user i has rated}$$

$$\psi_{ii'} = \text{set of movies both user i and i' have rated}$$

$$\psi_{ii'} = \psi_i \cap \psi_{i'}$$

Cosine similarity is another method for comparing vectors. Since we are using deviations, not averages, cosine similarity and Pearson correlation end up being equivalent.

## Computation Complexity

Ratings matrix R is an $N \times M$ matrix with N users and M items.

### Computing Similarity Between Two Users

$w(i, i')$ requires O(M) time to loop through all items

### Making a Prediction for a Single User

Even though we are only going to use the Top K users to compute item scores, we must compute similarity scores for all users to determine the top K.

There are N users, and each user similarity weight requires O(M). Therefore, computing similarity weights for all users requires O(NM) time.

### Making Predictions for All Users

We need $w(i, i')$ for $i = 1 \dots N$ and $i' = 1 \dots N$, i.e. $O(N^2)$ time.

Then we need $O(N^2M)$ time to make recommendations for all users.

### Making a Recommendation List for Single User

Calculate weights between user and all other users is O(MN).

Calculate scores s(i,j) for all items is also O(MN).

Sorting the scores is O(MlogM) time.

Total time is O(MN) + O(MlogM).

### Memory Consumption

Given 100K users, we need compute $100K \times 100K = 10^{10}$ user similarity weights. That's 10 billion.

If each weight is represented with 32 bit float (4 bytes), then we require 40 billion bytes $\approx$ 40GB. This exceeds most personal computers capabilities and will require large VMs or distributed systems.

### Working with a Subset

Always better to start with a subset of data and work out details. A good strategy is to:

- take top n users and top m movies (n < N, m < M)
- top users are users that rate the most items
- top items are the items with the most ratings


### Computing Recommendations Offline

Generally speaking, recommendation computations are too expensive to run in real time.

It's recommended to pre-compute user similarity weights, the Top K most similar users for each user, and/or the Top K best recommendations for a single user. These can be updated via cron jobs and stored in a DB. We lose realtime responsiveness, but we can then look up recommendations in O(1) time.

## Improving Performance

What if two users don't have anything in common, i.e. they have not rated any common movies? Or what if users have very little in common, e.g. near 0 correlation?

We should set a threshold, e.g. < 5, where we do not bother computing user similarity if they have less than 5 movies in common.

We should only consider a small neighborhood of users when computing item scores, computing summations over all users would be very expensive.

We can use the Top K users, e.g. 25 or 50 of the most similar users, to compute the weights. The most similar users are defined by users with highest weights. Since negative correlations can also be helpful, it may also be good to sort weights by their absolute value.

Computing user similarity weights is expensive. These can be learned offline and stored in DB for fast lookup. A reasonable strategy is to update these nightly.